In [1]:
import numpy as np
import pandas as pd
from collections import namedtuple, Counter
import copy


In [9]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from xgboost.sklearn import XGBClassifier
import pandas as pd
import pickle
import zipfile

In [69]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_regression
from sklearn.linear_model import LogisticRegression


def get_val(row,target,target_val):
    i=0
    while i<len(target):
        #print (row[target[i]],target_val[i])
        if not int(row[target[i]])==int(target_val[i]):
            return 0
        i+=1
    return 1

def get_prob_o_regression(df,conditional,conditional_values,target,target_val):
    new_lst=[]
    for index,row in df.iterrows():
        new_lst.append(get_val(row,target,target_val))
    
    X=df[conditional]
    regr = LogisticRegression(random_state=0)#RandomForestRegressor(max_depth=5, random_state=0)
    regr.fit(X, new_lst)
    return (regr.predict_proba([conditional_values])[0][1])
    #return(regr.predict([conditional_values])[0])
    

def get_logistic_param(df,conditional,target,target_val):
    new_lst=[]
    for index,row in df.iterrows():
        new_lst.append(get_val(row,target,target_val))
    
    X=df[conditional]
    regr = LogisticRegression(random_state=0)
    regr.fit(X, new_lst)
    return (regr.coef_.tolist()[0],list(regr.intercept_))
    #print(regr.get_params())

In [2]:
df=pd.read_csv('../data/loan_downsampled.csv')
feature = df.drop(['Target'], 1)
target = df[['Target']]

train_X, test_X, train_y, test_y = train_test_split(feature, target, random_state = 42)

In [12]:
model = XGBClassifier()
model = model.fit(train_X, train_y)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:98: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/anaconda3/lib/python3.8/site-packages/sklearn/preprocessing/_label.py:133: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[17:03:50] WARNING: /Users/runner/miniforge3/conda-bld/xgboost-split_1637426408905/work/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [13]:
pred_test_y = model.predict(test_X)
accuracy_score(pred_test_y, test_y)

0.6792878433899944

In [71]:
test_X['pred_test_y'] = pred_test_y

In [91]:
pred_test_y.mean()

0.5057600902279867

In [72]:
import math
from mip import Model, xsum, minimize, BINARY
def optimization(A,aval,Adomain,klst,kval,alpha,betalst,beta0):
    pogivenk=get_prob_o_regression(test_X,klst,kval,['pred_test_y'],[1])

    condition=copy.deepcopy(klst)
    condition.extend(A)
    conditionval=copy.deepcopy(kval)
    conditionval.extend(aval)
    popgivenak=get_prob_o_regression(test_X,condition,conditionval,['pred_test_y'],[0])
    pogivenak=get_prob_o_regression(test_X,condition,conditionval,['pred_test_y'],[1])
    pagivenk=get_prob_o_regression(df,klst,kval,A,aval)

    popagivenk=popgivenak*pagivenk
    
    alphak=pogivenak+alpha*popgivenak
    print (pogivenk,popagivenk)
    
    m = Model("Test")
    i=0
    var_lst=[]
    var_map={}
    while i<len(A):
        j=0
        while j<len(Adomain[i]):

            if Adomain[i][j]==aval[i]:
                j+=1
                continue

            var_lst.append(m.add_var(var_type=BINARY))
            var_map[len(var_lst)-1]=(i,j)
            j+=1
        i+=1
    print ("beta list is ",betalst,beta0)
    cost_lst=[]
    constr_lst=[]
    constr_lst.append(beta0)
    iter=0
    i=0
    while i<len(A):
        j=0
        del_cons=[]
        while j<len(Adomain[i]):

            if Adomain[i][j]==aval[i]:
                constr_lst.append(betalst[i]*Adomain[i][j])
                j+=1
                continue
            #print (j,"ASDSA",Adomain[i][j],aval[i],Adomain[i][j]==aval[i])
            cost_lst.append(var_lst[iter] * (Adomain[i][j]-aval[i]))#Cost is value - original value
            constr_lst.append(betalst[i]*var_lst[iter]*(Adomain[i][j]-aval[i]))
            del_cons.append(var_lst[iter])
            iter+=1
            j+=1
        m += xsum(del_cons) <= 1
        i+=1

    i=len(A)
    while i<len(betalst):
        constr_lst.append(betalst[i]*kval[i-len(A)])
        i+=1
    #print (constr_lst)
    print ("Constraint threshold",math.log(alphak*1.0/(1-alphak)),alphak)
    m+=xsum(constr_lst)>=math.log(alphak*1.0/(1-alphak))
    m.objective = minimize(xsum(cost_lst))
    m.optimize()
    if m.num_solutions:
        print('Objective value %g found:'
                  % (m.objective_value))
        i=0
        while i<len(var_lst):
            print (i,var_lst[i].x,var_map[i])
            i+=1
        
    

In [33]:
A=['FLAG_OWN_CAR: Y','FLAG_OWN_CAR: N', 
   'FLAG_OWN_REALTY: N', 'FLAG_OWN_REALTY: Y', 
   'NAME_TYPE_SUITE: Children', 'NAME_TYPE_SUITE: Family',
   'NAME_TYPE_SUITE: Group of people', 'NAME_TYPE_SUITE: Other_A',
   'NAME_TYPE_SUITE: Other_B', 'NAME_TYPE_SUITE: Spouse, partner',
   'NAME_TYPE_SUITE: Unaccompanied', 'NAME_TYPE_SUITE: nan',
   'WEEKDAY_APPR_PROCESS_START: FRIDAY',
   'WEEKDAY_APPR_PROCESS_START: MONDAY',
   'WEEKDAY_APPR_PROCESS_START: SATURDAY',
   'WEEKDAY_APPR_PROCESS_START: SUNDAY',
   'WEEKDAY_APPR_PROCESS_START: THURSDAY',
   'WEEKDAY_APPR_PROCESS_START: TUESDAY',
   'WEEKDAY_APPR_PROCESS_START: WEDNESDAY',
   'AMT_INCOME_TOTAL', 'AMT_CREDIT','AMT_ANNUITY',
   'FLAG_MOBIL', 'FLAG_EMP_PHONE','FLAG_WORK_PHONE', 
   'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL']

In [44]:
#choose index 452
# test_X[A][-1:].values
aval=[1.00000e+00, 0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 2.02500e+05,
        7.16427e+05, 5.67315e+04, 1.00000e+00, 1.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00]

In [48]:
klst = []
for i in feature.columns.values:
    if i not in A:
        klst.append(i)

In [88]:
#test_X[klst][-1:].values
kval = [ 1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  1.06830000e+05,
         1.00000000e+00,  6.48000000e+05,  2.51640000e-02,
        -1.32540000e+04, -1.74600000e+03, -1.72000000e+02,
        -4.64900000e+03,  3.00000000e+00,  2.00000000e+00,
         2.00000000e+00,  1.10000000e+01,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  2.68856030e-01,
         5.27265239e-04,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  5.00000000e+00]

In [75]:
Adomain = []
for i in A:
    Adomain.append(test_X[i].unique())

In [89]:


A=['FLAG_OWN_CAR: Y','FLAG_OWN_CAR: N', 
   'FLAG_OWN_REALTY: N', 'FLAG_OWN_REALTY: Y', 
   'NAME_TYPE_SUITE: Children', 'NAME_TYPE_SUITE: Family',
   'NAME_TYPE_SUITE: Group of people', 'NAME_TYPE_SUITE: Other_A',
   'NAME_TYPE_SUITE: Other_B', 'NAME_TYPE_SUITE: Spouse, partner',
   'NAME_TYPE_SUITE: Unaccompanied', 'NAME_TYPE_SUITE: nan',
   'WEEKDAY_APPR_PROCESS_START: FRIDAY',
   'WEEKDAY_APPR_PROCESS_START: MONDAY',
   'WEEKDAY_APPR_PROCESS_START: SATURDAY',
   'WEEKDAY_APPR_PROCESS_START: SUNDAY',
   'WEEKDAY_APPR_PROCESS_START: THURSDAY',
   'WEEKDAY_APPR_PROCESS_START: TUESDAY',
   'WEEKDAY_APPR_PROCESS_START: WEDNESDAY',
   'AMT_INCOME_TOTAL', 'AMT_CREDIT','AMT_ANNUITY',
   'FLAG_MOBIL', 'FLAG_EMP_PHONE','FLAG_WORK_PHONE', 
   'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL']#,'housing','employment']
aval=[0.00000e+00, 1.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.00000e+00, 0.00000e+00, 1.12500e+05,
        6.75000e+05, 2.19375e+04, 1.00000e+00, 0.00000e+00, 0.00000e+00,
        1.00000e+00, 0.00000e+00, 0.00000e+00]#,0,3] #len(aval)=len(A)

#klst=['sex','age','credit_history','property','month','purpose','foreign_worker']
#kval=[0,0,4,0,6,7,1]

conditionals=copy.deepcopy(A)
conditionals.extend(klst)
print (conditionals)
(beta_lst,[beta0])=get_logistic_param(test_X,conditionals,['pred_test_y'],[1])
print(beta_lst,beta0)



#Adomain=[[0, 1], []]#,[0,1],[0,1,2,3,4]]#List of list where each list is domain of each variable in A

optimization(A,aval,Adomain,klst,kval,0.819,beta_lst,beta0)

['FLAG_OWN_CAR: Y', 'FLAG_OWN_CAR: N', 'FLAG_OWN_REALTY: N', 'FLAG_OWN_REALTY: Y', 'NAME_TYPE_SUITE: Children', 'NAME_TYPE_SUITE: Family', 'NAME_TYPE_SUITE: Group of people', 'NAME_TYPE_SUITE: Other_A', 'NAME_TYPE_SUITE: Other_B', 'NAME_TYPE_SUITE: Spouse, partner', 'NAME_TYPE_SUITE: Unaccompanied', 'NAME_TYPE_SUITE: nan', 'WEEKDAY_APPR_PROCESS_START: FRIDAY', 'WEEKDAY_APPR_PROCESS_START: MONDAY', 'WEEKDAY_APPR_PROCESS_START: SATURDAY', 'WEEKDAY_APPR_PROCESS_START: SUNDAY', 'WEEKDAY_APPR_PROCESS_START: THURSDAY', 'WEEKDAY_APPR_PROCESS_START: TUESDAY', 'WEEKDAY_APPR_PROCESS_START: WEDNESDAY', 'AMT_INCOME_TOTAL', 'AMT_CREDIT', 'AMT_ANNUITY', 'FLAG_MOBIL', 'FLAG_EMP_PHONE', 'FLAG_WORK_PHONE', 'FLAG_CONT_MOBILE', 'FLAG_PHONE', 'FLAG_EMAIL', 'NAME_CONTRACT_TYPE: Cash loans', 'NAME_CONTRACT_TYPE: Revolving loans', 'CODE_GENDER: F', 'CODE_GENDER: M', 'CODE_GENDER: XNA', 'NAME_INCOME_TYPE: Businessman', 'NAME_INCOME_TYPE: Commercial associate', 'NAME_INCOME_TYPE: Maternity leave', 'NAME_INCOME

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


[-3.67658528043389e-08, 1.1854505999445792e-07, 5.837273382435295e-08, 2.3406473365761032e-08, -2.624206178981828e-09, -9.455293478262197e-09, 4.4932435881506407e-10, 1.9085394116189523e-11, 1.1063790085179082e-09, 1.4128167428232055e-09, 8.973983118752189e-08, 1.1312701555701821e-09, 2.9274778279785105e-08, 1.9942084240273466e-09, 4.669731673350233e-10, 1.1473100332099736e-08, 1.536936135304432e-08, 1.7457719550607077e-08, 5.743066083218279e-09, -3.348626679766912e-07, 4.194281239459295e-06, 2.8925429901627794e-05, 8.177920719011575e-08, 8.341938086115943e-08, 7.576880192930342e-08, 8.152855804991557e-08, -2.3256511098956438e-08, -1.8236059147325938e-09, 1.0467643928094869e-07, -2.28972320908315e-08, -1.0047912000407473e-07, 1.8225832719418703e-07, 0.0, 0.0, -3.7040996111640695e-08, 2.613618958867621e-10, -2.4058155955889693e-09, -3.993837665950678e-08, 0.0, 7.65641924547299e-10, 1.6013739173639667e-07, -3.1291444667577627e-10, -1.5362856377377236e-07, 3.7349042415355414e-10, 1.125715

/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:445: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to 

0.4831506206266365 8.382621219499798e-05
beta list is  [-3.67658528043389e-08, 1.1854505999445792e-07, 5.837273382435295e-08, 2.3406473365761032e-08, -2.624206178981828e-09, -9.455293478262197e-09, 4.4932435881506407e-10, 1.9085394116189523e-11, 1.1063790085179082e-09, 1.4128167428232055e-09, 8.973983118752189e-08, 1.1312701555701821e-09, 2.9274778279785105e-08, 1.9942084240273466e-09, 4.669731673350233e-10, 1.1473100332099736e-08, 1.536936135304432e-08, 1.7457719550607077e-08, 5.743066083218279e-09, -3.348626679766912e-07, 4.194281239459295e-06, 2.8925429901627794e-05, 8.177920719011575e-08, 8.341938086115943e-08, 7.576880192930342e-08, 8.152855804991557e-08, -2.3256511098956438e-08, -1.8236059147325938e-09, 1.0467643928094869e-07, -2.28972320908315e-08, -1.0047912000407473e-07, 1.8225832719418703e-07, 0.0, 0.0, -3.7040996111640695e-08, 2.613618958867621e-10, -2.4058155955889693e-09, -3.993837665950678e-08, 0.0, 7.65641924547299e-10, 1.6013739173639667e-07, -3.1291444667577627e-10, -1

In [ ]:

A=['status','credit_history']#,'housing','employment']
aval=[0,0]

klst=['sex','age','property','month','purpose','foreign_worker']
kval=[0,0,0,6,7,1]

conditionals=copy.deepcopy(A)
conditionals.extend(klst)
print (conditionals)
(beta_lst,[beta0])=get_logistic_param(df,conditionals,['credit'],[1])
print(beta_lst,beta0)



Adomain=[[0,1,2,3],[0,1,2,3,4,5,6]]#,[0,1],[0,1,2,3,4]]#List of list where each list is domain of each variable in A

optimization(A,aval,Adomain,klst,kval,0.9,beta_lst,beta0)

In [ ]:
##Additional test to debug

In [111]:
import math
i=0
score=beta0
while i<len(klst):
    score+=kval[i]*beta_lst[i+1]
    i+=1
    
score+=beta_lst[0]*3
print(score,1.0/(1+math.exp(-score)))

3.5646574618886584 0.9724725313054886


In [112]:
condition=[]
condition.extend(A)
condition.extend(klst)#copy.deepcopy(klst)

conditionval=[]
conditionval.extend([3])
conditionval.extend(kval)
print (condition)
pogivenak=get_prob_o_regression(df,condition,conditionval,['credit'],[1])
    

['status', 'sex', 'age', 'credit_history', 'property', 'month', 'purpose', 'foreign_worker']


In [113]:
klst,kval,A

(['sex',
  'age',
  'credit_history',
  'property',
  'month',
  'purpose',
  'foreign_worker'],
 [0, 0, 4, 0, 6, 7, 1],
 ['status'])

In [114]:
pogivenak

0.9724725313054886

In [82]:
condition=copy.deepcopy(klst)
condition.extend(A)
conditionval=copy.deepcopy(kval)
conditionval.extend([0])
pogivenaoldk=get_prob_o_regression(df,condition,conditionval,['credit'],[1])
    

In [83]:
pogivenaoldk

0.7408379839662487

In [84]:
(pogivenak-pogivenaoldk)*1.0/(1-pogivenaoldk)

0.7223711655101134